# Performance TensorFlow==2.1.0 Software Stack Combinations Errors

The purpose of this notebook is to show the errors present in software stacks during run using PIimport.

## Inputs for Dataset created using Amun

All data have been gathered using [Amun Service](https://github.com/thoth-station/amun-api) and [Performance Indicators](https://github.com/thoth-station/performance) evaluated by Thoth Team.


## TensorFlow builds

Tensorflow builds have been created considering combinations of the following parameters:

**Software stacks and native dependencies**

All inspections use a combination of all stacks from the dependencies of TensorFlow in version 2.1.0. 


  * `upstream TensorFlow` - `tensorflow==2.1.0` available on PyPI (inspections prefixed with `tf`)

**OS images**

  * `rhel-8` 

**Python Interpreters**

  * `3.6` 
  
**Hardware**

No node pinning used, any hardware available on OCP is used. No GPU was used. 
Analysis across inspection run will show which hardware have been identified.

`Number of CPUs` used to run is selected a priori as input to Amun: 1

## Performance indicators
Performance Indicators (PI) used for performance analysis:

  * [import](https://github.com/thoth-station/performance/blob/master/tensorflow/import.py)

Each performance indicator was run `1 times` per inspection run (`batch size == 1`), performance indicators reported median of inspections to be further compared.

## Dataset content

Inspection specification, build logs, job logs, hardware information of the node where the performance indicator was run and the actual inspection job result are included in the dataset.

No buildtime errors spotted with the tested stack.

There are some runtime errors spotted with specific stack.


## Analysis

Analysis show which versions exactly failed during run, so that Thoth can discard packages failing during run.

## Assign environment variables and import libraries

In [2]:
%env THOTH_DEPLOYMENT_NAME     ocp-stage
%env THOTH_CEPH_HOST           https://s3.upshift.redhat.com/
%env THOTH_CEPH_BUCKET         thoth
%env THOTH_CEPH_BUCKET_PREFIX  data

env: THOTH_DEPLOYMENT_NAME=ocp-stage
env: THOTH_CEPH_HOST=https://s3.upshift.redhat.com/
env: THOTH_CEPH_BUCKET=thoth
env: THOTH_CEPH_BUCKET_PREFIX=data


In [3]:
from thoth.report_processing.components.inspection import AmunInspections
from thoth.report_processing.components.inspection import AmunInspectionsSummary
from thoth.report_processing.components.inspection import AmunInspectionsStatistics
from thoth.report_processing.components.inspection import AmunInspectionsFailedSummary

inspection = AmunInspections()
inspection_runs_summary = AmunInspectionsSummary()
inspection_statistics = AmunInspectionsStatistics()
inspection_failed_summary = AmunInspectionsFailedSummary()

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', 400)
pd.options.plotting.backend = "plotly"  # Convert to matplotlib

In [ ]:
FILE_NAME = "thoth-performance-dataset-tf2.1.0-combinations-errors.zip"

from thoth.report_processing.utils import extract_zip_file

extract_zip_file(FILE_NAME)

In [6]:
inspections_identifiers = ['tf-dm-six']  # List of identifiers for the analysis

## Retrieve and process data

In [ ]:
from pathlib import Path

current_path = Path.cwd()

inspection_runs = inspection.aggregate_thoth_inspections_results(
    inspections_identifiers=inspections_identifiers,
    is_local=True,
    repo_path=current_path.joinpath("inspections-tf-dm-six")
)

In [ ]:
processed_inspection_runs, failed_inspection_runs = inspection.process_inspection_runs(
    inspection_runs,
)

In [ ]:
inspections_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=processed_inspection_runs,
    include_statistics=True
)

In [12]:
inspections_df.head()

,end_datetime,exit_code,hostname,script_sha256,start_datetime,stderr,inspection_document_id,identifier,specification_base,batch_size,hwinfo__cpu_features__flags,hwinfo__cpu_features__hz_actual,hwinfo__cpu_features__hz_actual_raw,hwinfo__cpu_features__hz_advertised,hwinfo__cpu_features__hz_advertised_raw,hwinfo__cpu_features__l1_data_cache_size,hwinfo__cpu_features__l1_instruction_cache_size,hwinfo__cpu_features__l2_cache_size,hwinfo__cpu_features__l3_cache_size,hwinfo__cpu_info__arch,hwinfo__cpu_info__arch_string_raw,hwinfo__cpu_info__bits,hwinfo__cpu_info__brand_raw,hwinfo__cpu_info__count,hwinfo__cpu_info__cpuinfo_version,hwinfo__cpu_info__cpuinfo_version_string,hwinfo__cpu_info__family,hwinfo__cpu_info__hz_actual_friendly,hwinfo__cpu_info__hz_advertised_friendly,hwinfo__cpu_info__l2_cache_associativity,hwinfo__cpu_info__l2_cache_line_size,hwinfo__cpu_info__model,hwinfo__cpu_info__stepping,hwinfo__cpu_info__vendor_id_raw,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU,hwinfo__cpu_type__nbits,hwinfo__cpu_type__ncpus,hwinfo__cpu_type__not_impl,hwinfo__cpu_type__try_call,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version,os_release__ansi_color,os_release__bug_report_url,os_release__cpe_name,os_release__home_url,os_release__id,os_release__id_like,os_release__name,os_release__platform_id,os_release__pretty_name,os_release__redhat_bugzilla_product,os_release__redhat_bugzilla_product_version,os_release__redhat_support_product,os_release__redhat_support_product_version,os_release__version,os_release__version_id,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model,runtime_environment__operating_system__name,runtime_environment__operating_system__version,runtime_environment__python_version,stdout__@result__elapsed,stdout__component,stdout__name,stdout__tensorflow_aicoe_buildinfo,stdout__tensorflow_upstream_buildinfo,usage__ru_idrss,usage__ru_inblock,usage__ru_isrss,usage__ru_ixrss,usage__ru_majflt,usage__ru_maxrss,usage__ru_minflt,usage__ru_msgrcv,usage__ru_msgsnd,usage__ru_nivcsw,usage__ru_nsignals,usage__ru_nswap,usage__ru_nvcsw,usage__ru_oublock,usage__ru_stime,usage__ru_utime,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requirements_locked___meta__hash__sha256,requirements_locked___meta__pipfile-spec,requirements_locked__default__tensorflow__version,requirements_locked__default__tensorflow_

# Inspections summary report

In [13]:
report_results, _ = inspection_runs_summary.produce_summary_report(inspections_df=inspections_df)

## Hardware

In [14]:
report_results["hardware"]['platform'].head()

,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version
0,nan,x86_64,inspection-tf-dm-six-96c3d527-2695816303,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-redhat-8.2-Ootpa,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
1,nan,x86_64,inspection-tf-dm-six-7f53caaa-3451135753,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-redhat-8.2-Ootpa,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
2,nan,x86_64,inspection-tf-dm-six-0d800caa-447756917,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-redhat-8.2-Ootpa,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
3,nan,x86_64,inspection-tf-dm-six-d9316be5-3840106319,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-redhat-8.2-Ootpa,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020
4,nan,x86_64,inspection-tf-dm-six-64c60ffa-3406776632,Linux-4.18.0-147.8.1.el8_1.x86_64-x86_64-with-redhat-8.2-Ootpa,x86_64,4.18.0-147.8.1.el8_1.x86_64,#1 SMP Wed Feb 26 03:08:15 UTC 2020


In [15]:
report_results["hardware"]['processor']

,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU
0,False,False,nan,nan,nan,True,True,True,True,True,False,True,False,False,nan,False,False,False,False,False,False,False,False,False,nan,nan,nan,False,True,nan,False,False,nan,False,False,False,False,False,False,False,nan,nan,nan,nan,False,True,True,nan,nan,False,True,False


In [16]:
report_results["hardware"]['flags']

,flag__3dnowprefetch,flag__abm,flag__adx,flag__aes,flag__apic,flag__arat,flag__avx,flag__avx2,flag__avx512bw,flag__avx512cd,flag__avx512dq,flag__avx512f,flag__avx512vl,flag__bmi1,flag__bmi2,flag__clflush,flag__clflushopt,flag__clwb,flag__cmov,flag__constant_tsc,flag__cpuid,flag__cx16,flag__cx8,flag__de,flag__ept,flag__erms,flag__f16c,flag__flexpriority,flag__fma,flag__fpu,flag__fsgsbase,flag__fxsr,flag__hle,flag__hypervisor,flag__ibpb,flag__ibrs,flag__invpcid,flag__invpcid_single,flag__lahf_lm,flag__lm,flag__mca,flag__mce,flag__md_clear,flag__mmx,flag__movbe,flag__mpx,flag__msr,flag__mtrr,flag__nopl,flag__nx,flag__ospke,flag__osxsave,flag__pae,flag__pat,flag__pcid,flag__pclmulqdq,flag__pdpe1gb,flag__pge,flag__pku,flag__pni,flag__popcnt,flag__pse,flag__pse36,flag__pti,flag__rdrand,flag__rdrnd,flag__rdseed,flag__rdtscp,flag__rep_good,flag__rtm,flag__sep,flag__smap,flag__smep,flag__ss,flag__ssbd,flag__sse,flag__sse2,flag__sse4_1,flag__sse4_2,flag__ssse3,flag__stibp,flag__syscall,flag__tpr_shadow,flag__tsc,flag__tsc_adjust,flag__tsc_deadline_timer,flag__tsc_known_freq,flag__tscdeadline,flag__vme,flag__vmx,flag__vnmi,flag__vpid,flag__x2apic,flag__xgetbv1,flag__xsave,flag__xsavec,flag__xsaveopt,flag__xtopology
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [17]:
report_results["hardware"]['ncpus']

,hwinfo__cpu_type__ncpus
0,64


In [18]:
report_results["hardware"]['info']

,hwinfo__cpu_info__brand_raw,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model
0,"Intel Xeon Processor (Skylake, IBRS)",nan,6,85


## Operating System

In [19]:
report_results["base_image"]['base_image']

,specification_base,os_release__name,os_release__version,os_release__version_id
0,quay.io/thoth-station/s2i-thoth-ubi8-py36,Red Hat Enterprise Linux,8.2 (Ootpa),8.2


In [20]:
report_results["base_image"]['number_cpus_run']

,run__requests__cpu
0,1


## Performance Indicator

In [21]:
report_results["pi"]['pi']

,script_sha256,batch_size,stdout__component,stdout__name
0,8e0ad2b4cac88850e268b9d7b72b8569e564ac2ac106cf1d486380c93811c695,1,tensorflow,PiImport


## Software Stack

In [30]:
report_results["software_stack"]['requirements_locked'].head()

,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requirements_locked___meta__hash__sha256,requirements_locked___meta__pipfile-spec,requirements_locked__default__tensorflow__version,requirements_locked__default__tensorflow__index,requirements_locked__default__absl-py__version,requirements_locked__default__absl-py__index,requirements_locked__default__astor__version,requirements_locked__default__astor__index,requirements_locked__default__gast__version,requirements_locked__default__gast__index,requirements_locked__default__google-pasta__version,requirements_locked__default__google-pasta__index,requirements_locked__default__grpcio__version,requirements_locked__default__grpcio__index,requirements_locked__default__keras-applications__version,requirements_locked__default__keras-applications__index,requirements_locked__default__keras-preprocessing__version,requirements_locked__default__keras-preprocessing__index,requirements_locked__default__numpy__version,requirements_locked__default__numpy__index,requirements_locked__default__opt-einsum__version,requirements_locked__default__opt-einsum__index,requirements_locked__default__protobuf__version,requirements_locked__default__protobuf__index,requirements_locked__default__scipy__version,requirements_locked__default__scipy__index,requirements_locked__default__tensorboard__version,requirements_locked__default__tensorboard__index,requirements_locked__default__tensorflow-estimator__version,requirements_locked__default__tensorflow-estimator__index,requirements_locked__default__termcolor__version,requirements_locked__default__termcolor__index,requirements_locked__default__wheel__version,requirements_locked__default__wheel__index,requirements_locked__default__wrapt__version,requirements_locked__default__wrapt__index,requirements_locked__default__h5py__version,requirements_locked__default__h5py__index,requirements_locked__default__setuptools__version,requirements_locked__default__setuptools__index,requirements_locked__default__google-auth__version,requirements_locked__default__google-auth__index,requirements_locked__default__google-auth-oauthlib__version,requirements_locked__default__google-auth-oauthlib__index,requirements_locked__default__markdown__version,requirements_locked__default__markdown__index,requirements_locked__default__requests__version,requirements_locked__default__requests__index,requirements_locked__default__werkzeug__version,requirements_locked__default__werkzeug__index,requirements_locked__default__cachetools__version,requirements_locked__default__cachetools__index,requirements_locked__default__pyasn1-modules__version,requirements_locked__default__pyasn1-modules__index,requirements_locked__default__rsa__version,requirements_locked__default__rsa__index,requirements_locked__default__requests-oauthlib__version,requirements_locked__default__requests-oauthlib__index,requirements_locked__default__certifi__version,requirements_locked__default__certifi__index,requirements_locked__default__chardet__version,requirements_locked__default__chardet__index,requirements_locked__default__idna__version,requirements_locked__default__idna__index,requirements_locked__default__pyasn1__version,requirements_locked__default__pyasn1__index,requirements_locked__default__oauthlib__version,requirements_locked__default__oauthlib__index,requirements_locked__default__six__version,requirements_locked__default__six__index,requirements_locked__default__urllib3__version,requirements_locked__default__urllib3__index
0,nan,3.6,c9e7f586cc7f527664b35f4581b3d5421512489efb2c89f19b2064a9eb308c1d,6,==2.1.0,pypi-org,==0.8.0,pypi-org,==0.8.0,pypi-org,==0.2.2,pypi-org,==0.1.8,pypi-org,==1.25.0,pypi-org,==1.0.8,pypi-org,==1.1.0,pypi-org,==1.17.4,pypi-org,==3.1.0,pypi-org,==3.11.1,pypi-org,==1.4.1,pypi-org,==2.1.0,pypi-org,==2.1.0,pypi-org,==1.1.0,pypi-org,==0.33.6,pypi-org,==1.11.2,pypi-org,==2.10.0,pypi-org,==42.0.2,pypi-org,==1.7.0,pypi-org,==0.4.1,pypi-org,==3.1.1,pypi-org,==2.21.0,pyp

In [29]:
python_packages_dataframe, python_packages_versions = inspection.create_python_package_df(inspections_df=inspections_df)
python_packages_dataframe.head()

,tensorflow,absl-py,astor,gast,google-pasta,grpcio,keras-applications,keras-preprocessing,numpy,opt-einsum,protobuf,scipy,tensorboard,tensorflow-estimator,termcolor,wheel,wrapt,h5py,setuptools,google-auth,google-auth-oauthlib,markdown,requests,werkzeug,cachetools,pyasn1-modules,rsa,requests-oauthlib,certifi,chardet,idna,pyasn1,oauthlib,six,urllib3
0,tensorflow-2.1.0-pypi-org,absl-py-0.8.0-pypi-org,astor-0.8.0-pypi-org,gast-0.2.2-pypi-org,google-pasta-0.1.8-pypi-org,grpcio-1.25.0-pypi-org,keras-applications-1.0.8-pypi-org,keras-preprocessing-1.1.0-pypi-org,numpy-1.17.4-pypi-org,opt-einsum-3.1.0-pypi-org,protobuf-3.11.1-pypi-org,scipy-1.4.1-pypi-org,tensorboard-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,termcolor-1.1.0-pypi-org,wheel-0.33.6-pypi-org,wrapt-1.11.2-pypi-org,h5py-2.10.0-pypi-org,setuptools-42.0.2-pypi-org,google-auth-1.7.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,markdown-3.1.1-pypi-org,requests-2.21.0-pypi-org,werkzeug-0.16.0-pypi-org,cachetools-3.1.1-pypi-org,pyasn1-modules-0.2.7-pypi-org,rsa-4.0-pypi-org,requests-oauthlib-1.3.0-pypi-org,certifi-2019.11.28-pypi-org,chardet-3.0.4-pypi-org,idna-2.8-pypi-org,pyasn1-0.4.8-pypi-org,oauthlib-3.1.0-pypi-org,six-1.13.0-pypi-org,urllib3-1.14-pypi-org
1,tensorflow-2.1.0-pypi-org,absl-py-0.8.0-pypi-org,astor-0.8.0-pypi-org,gast-0.2.2-pypi-org,google-pasta-0.1.8-pypi-org,grpcio-1.25.0-pypi-org,keras-applications-1.0.8-pypi-org,keras-preprocessing-1.1.0-pypi-org,numpy-1.17.4-pypi-org,opt-einsum-3.1.0-pypi-org,protobuf-3.11.1-pypi-org,scipy-1.4.1-pypi-org,tensorboard-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,termcolor-1.1.0-pypi-org,wheel-0.33.6-pypi-org,wrapt-1.11.2-pypi-org,h5py-2.10.0-pypi-org,setuptools-42.0.2-pypi-org,google-auth-1.7.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,markdown-3.1.1-pypi-org,requests-2.21.0-pypi-org,werkzeug-0.16.0-pypi-org,cachetools-3.1.1-pypi-org,pyasn1-modules-0.2.7-pypi-org,rsa-4.0-pypi-org,requests-oauthlib-1.3.0-pypi-org,certifi-2019.11.28-pypi-org,chardet-3.0.4-pypi-org,idna-2.8-pypi-org,pyasn1-0.4.8-pypi-org,oauthlib-3.1.0-pypi-org,six-1.13.0-pypi-org,urllib3-1.12-pypi-org
2,tensorflow-2.1.0-pypi-org,absl-py-0.8.0-pypi-org,astor-0.8.0-pypi-org,gast-0.2.2-pypi-org,google-pasta-0.1.8-pypi-org,grpcio-1.25.0-pypi-org,keras-applications-1.0.8-pypi-org,keras-preprocessing-1.1.0-pypi-org,numpy-1.17.4-pypi-org,opt-einsum-3.1.0-pypi-org,protobuf-3.11.1-pypi-org,scipy-1.4.1-pypi-org,tensorboard-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,termcolor-1.1.0-pypi-org,wheel-0.33.6-pypi-org,wrapt-1.11.2-pypi-org,h5py-2.10.0-pypi-org,setuptools-42.0.2-pypi-org,google-auth-1.7.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,markdown-3.1.1-pypi-org,requests-2.22.0-pypi-org,werkzeug-0.16.0-pypi-org,cachetools-3.1.1-pypi-org,pyasn1-modules-0.2.7-pypi-org,rsa-4.0-pypi-org,requests-oauthlib-1.3.0-pypi-org,certifi-2019.11.28-pypi-org,chardet-3.0.4-pypi-org,idna-2.8-pypi-org,pyasn1-0.4.8-pypi-org,oauthlib-3.1.0-pypi-org,six-1.13.0-pypi-org,urllib3-1.22-pypi-org
3,tensorflow-2.1.0-pypi-org,absl-py-0.8.0-pypi-org,astor-0.8.0-pypi-org,gast-0.2.2-pypi-org,google-pasta-0.1.8-pypi-org,grpcio-1.25.0-pypi-org,keras-applications-1.0.8-pypi-org,keras-preprocessing-1.1.0-pypi-org,numpy-1.17.4-pypi-org,opt-einsum-3.1.0-pypi-org,protobuf-3.11.1-pypi-org,scipy-1.4.1-pypi-org,tensorboard-2.1.0-pypi-org,tensorflow-estimator-2.1.0-pypi-org,termcolor-1.1.0-pypi-org,wheel-0.33.6-pypi-org,wrapt-1.11.2-pypi-org,h5py-2.10.0-pypi-org,setuptools-42.0.2-pypi-org,google-auth-1.7.0-pypi-org,google-auth-oauthlib-0.4.1-pypi-org,markdown-3.1.1-pypi-org,requests-2.21.0-pypi-org,werkzeug-0.16.0-pypi-org,cachetools-3.1.1-pypi-org,pyasn1-modules-0.2.7-pypi-org,rsa-4.0-pypi-org,requests-oauthlib-1.3.0-pypi-org,certifi-2019.11.28-pypi-org,chardet-3.0.4-pypi-org,idna-2.8-pypi-org,pyasn1-0.4.8-pypi-org,oauthlib-3.1.0-pypi-org,six-1.13.0-pypi-org,urllib3-1.7-pypi-org
4,tensorflow-2.1.0-pypi-org,absl-py-0.8.0-pypi-org,astor-0.8.0-pypi-org,gast-0.2.2-pypi-org,google-pasta-0.

# Failed Inspection Summary

In [24]:
failed_inspections_df = inspection.create_inspections_dataframe(
    processed_inspection_runs=failed_inspection_runs,
)

In [25]:
failed_inspections_df.head()

,end_datetime,exit_code,hostname,script_sha256,start_datetime,stderr,inspection_document_id,identifier,specification_base,batch_size,hwinfo__cpu_features__flags,hwinfo__cpu_features__hz_actual,hwinfo__cpu_features__hz_actual_raw,hwinfo__cpu_features__hz_advertised,hwinfo__cpu_features__hz_advertised_raw,hwinfo__cpu_features__l1_data_cache_size,hwinfo__cpu_features__l1_instruction_cache_size,hwinfo__cpu_features__l2_cache_size,hwinfo__cpu_features__l3_cache_size,hwinfo__cpu_info__arch,hwinfo__cpu_info__arch_string_raw,hwinfo__cpu_info__bits,hwinfo__cpu_info__brand_raw,hwinfo__cpu_info__count,hwinfo__cpu_info__cpuinfo_version,hwinfo__cpu_info__cpuinfo_version_string,hwinfo__cpu_info__family,hwinfo__cpu_info__hz_actual_friendly,hwinfo__cpu_info__hz_advertised_friendly,hwinfo__cpu_info__l2_cache_associativity,hwinfo__cpu_info__l2_cache_line_size,hwinfo__cpu_info__model,hwinfo__cpu_info__stepping,hwinfo__cpu_info__vendor_id_raw,hwinfo__cpu_type__has_3dnow,hwinfo__cpu_type__has_3dnowext,hwinfo__cpu_type__has_Altivec,hwinfo__cpu_type__has_f00f_bug,hwinfo__cpu_type__has_fdiv_bug,hwinfo__cpu_type__has_mmx,hwinfo__cpu_type__has_sse,hwinfo__cpu_type__has_sse2,hwinfo__cpu_type__has_sse3,hwinfo__cpu_type__has_ssse3,hwinfo__cpu_type__is_32bit,hwinfo__cpu_type__is_64bit,hwinfo__cpu_type__is_AMD,hwinfo__cpu_type__is_AMD64,hwinfo__cpu_type__is_Alpha,hwinfo__cpu_type__is_Athlon64,hwinfo__cpu_type__is_AthlonHX,hwinfo__cpu_type__is_AthlonK6,hwinfo__cpu_type__is_AthlonK6_2,hwinfo__cpu_type__is_AthlonK6_3,hwinfo__cpu_type__is_AthlonK7,hwinfo__cpu_type__is_AthlonMP,hwinfo__cpu_type__is_Celeron,hwinfo__cpu_type__is_Core2,hwinfo__cpu_type__is_EV4,hwinfo__cpu_type__is_EV5,hwinfo__cpu_type__is_EV56,hwinfo__cpu_type__is_Hammer,hwinfo__cpu_type__is_Intel,hwinfo__cpu_type__is_Itanium,hwinfo__cpu_type__is_Nocona,hwinfo__cpu_type__is_Opteron,hwinfo__cpu_type__is_PCA56,hwinfo__cpu_type__is_Pentium,hwinfo__cpu_type__is_PentiumII,hwinfo__cpu_type__is_PentiumIII,hwinfo__cpu_type__is_PentiumIV,hwinfo__cpu_type__is_PentiumM,hwinfo__cpu_type__is_PentiumMMX,hwinfo__cpu_type__is_PentiumPro,hwinfo__cpu_type__is_Power,hwinfo__cpu_type__is_Power7,hwinfo__cpu_type__is_Power8,hwinfo__cpu_type__is_Power9,hwinfo__cpu_type__is_Prescott,hwinfo__cpu_type__is_XEON,hwinfo__cpu_type__is_Xeon,hwinfo__cpu_type__is_i386,hwinfo__cpu_type__is_i486,hwinfo__cpu_type__is_i586,hwinfo__cpu_type__is_i686,hwinfo__cpu_type__is_singleCPU,hwinfo__cpu_type__nbits,hwinfo__cpu_type__ncpus,hwinfo__cpu_type__not_impl,hwinfo__cpu_type__try_call,hwinfo__platform__architecture,hwinfo__platform__machine,hwinfo__platform__node,hwinfo__platform__platform,hwinfo__platform__processor,hwinfo__platform__release,hwinfo__platform__version,os_release__ansi_color,os_release__bug_report_url,os_release__cpe_name,os_release__home_url,os_release__id,os_release__id_like,os_release__name,os_release__platform_id,os_release__pretty_name,os_release__redhat_bugzilla_product,os_release__redhat_bugzilla_product_version,os_release__redhat_support_product,os_release__redhat_support_product_version,os_release__version,os_release__version_id,runtime_environment__cuda_version,runtime_environment__hardware__cpu_family,runtime_environment__hardware__cpu_model,runtime_environment__operating_system__name,runtime_environment__operating_system__version,runtime_environment__python_version,usage__ru_idrss,usage__ru_inblock,usage__ru_isrss,usage__ru_ixrss,usage__ru_majflt,usage__ru_maxrss,usage__ru_minflt,usage__ru_msgrcv,usage__ru_msgsnd,usage__ru_nivcsw,usage__ru_nsignals,usage__ru_nswap,usage__ru_nvcsw,usage__ru_oublock,usage__ru_stime,usage__ru_utime,requirements_locked___meta__sources,requirements_locked___meta__requires__python_version,requirements_locked___meta__hash__sha256,requirements_locked___meta__pipfile-spec,requirements_locked__default__tensorflow__version,requirements_locked__default__tensorflow__hashes,requirements_locked__default__tensorflow__index,requirements_locked__default__absl-py__version,requirements_locked__defau

In [27]:
comparison_df = inspection_failed_summary.show_software_stack_differences(
    inspections_df,
    failed_inspections_df
)
comparison_df

,package,versions_in_successfull,versions_in_failed,versions_in_failed_only,versions_common
0,absl-py,[absl-py-0.8.0-pypi-org],[absl-py-0.8.0-pypi-org],[],[absl-py-0.8.0-pypi-org]
1,astor,[astor-0.8.0-pypi-org],[astor-0.8.0-pypi-org],[],[astor-0.8.0-pypi-org]
2,cachetools,[cachetools-3.1.1-pypi-org],[cachetools-3.1.1-pypi-org],[],[cachetools-3.1.1-pypi-org]
3,certifi,[certifi-2019.11.28-pypi-org],[certifi-2019.11.28-pypi-org],[],[certifi-2019.11.28-pypi-org]
4,chardet,[chardet-3.0.4-pypi-org],[chardet-3.0.4-pypi-org],[],[chardet-3.0.4-pypi-org]
5,gast,[gast-0.2.2-pypi-org],[gast-0.2.2-pypi-org],[],[gast-0.2.2-pypi-org]
6,google-auth,[google-auth-1.7.0-pypi-org],[google-auth-1.7.0-pypi-org],[],[google-auth-1.7.0-pypi-org]
7,google-auth-oauthlib,[google-auth-oauthlib-0.4.1-pypi-org],[google-auth-oauthlib-0.4.1-pypi-org],[],[google-auth-oauthlib-0.4.1-pypi-org]
8,google-pasta,[google-pasta-0.1.8-pypi-org],[google-pasta-0.1.8-pypi-org],[],[google-pasta-0.1.8-pypi-org]
9,grpcio,[grpcio-1.25.0-pypi-org],[grpcio-1.25.0-pypi-org],[],[grpcio-1.25.0-pypi-org]


In [28]:
comparison_df[comparison_df['package'].isin(['six', 'urllib3'])]

,package,versions_in_successfull,versions_in_failed,versions_in_failed_only,versions_common
26,six,"[six-1.12.0-pypi-org, six-1.13.0-pypi-org]",[six-1.13.0-pypi-org],[],[six-1.13.0-pypi-org]
31,urllib3,"[urllib3-0.3-pypi-org, urllib3-1.10-pypi-org, urllib3-1.10.1-pypi-org, urllib3-1.10.2-pypi-org, urllib3-1.10.3-pypi-org, urllib3-1.10.4-pypi-org, urllib3-1.11-pypi-org, urllib3-1.12-pypi-org, urllib3-1.13-pypi-org, urllib3-1.13.1-pypi-org, urllib3-1.14-pypi-org, urllib3-1.15-pypi-org, urllib3-1.15.1-pypi-org, urllib3-1.16-pypi-org, urllib3-1.17-pypi-org, urllib3-1.18-pypi-org, urllib3-1.18.1-p...","[urllib3-1.2-pypi-org, urllib3-1.2.2-pypi-org, urllib3-1.3-pypi-org, urllib3-1.4-pypi-org, urllib3-1.5-pypi-org]","[urllib3-1.2-pypi-org, urllib3-1.2.2-pypi-org, urllib3-1.3-pypi-org, urllib3-1.4-pypi-org, urllib3-1.5-pypi-org]",[]
